# This notebook is to prepare the data to pass to the saved SVD model to make predictions

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Get ratings table and find all unique venues in the table that have been rated.

In [ ]:
ratingsdf = pd.read_csv('../input/ml-challenge/ratings.csv')
ratingsdf.count()

In [ ]:
ratingsdf['venue_id'].nunique()

In [ ]:
uniqueVenuedf = ratingsdf['venue_id']
uniqueVenuedf.drop_duplicates(inplace=True)
uniqueVenuedf.count()

In [ ]:
uniqueVenuedf.head()

In [ ]:
venuesdf = pd.read_csv('../input/data-exploration/correct_venues.csv')
venuesdf.count()


In [ ]:
result = pd.merge(venuesdf,uniqueVenuedf, left_on='id', right_on='venue_id')
result = result.drop('venue_id', axis=1)
result.head(20)

In [ ]:
result.shape

Now we need to combine with the top 10 user table with their locations

In [ ]:
top10df = pd.read_csv('../input/data-exploration/top10checkins.csv')
top10df.head()

In [ ]:
result['tmp'] = 1
top10df['tmp'] = 1
combinedf = pd.merge(top10df, result, on=['tmp'])
combinedf = combinedf.drop('tmp', axis=1)
combinedf.head()

In [ ]:
combinedf.shape

Wow, the potential list of venues for the top 10 check-in users has almost 16 million rows!
We need to reduce this list before passing to the SVD prediction model.

One way to do that would be for each user, we should only shortlist the venues that are within a certain distance, let's say 80km (50 miles) from the user's location, to reflect the likelihood that a user isn't going to spend hours to travel to a faraway venue even though it's highly rated.

Due to Kaggle's session timing limits, I will need to run several notebooks to reduce this list, so let's go ahead to save this large dataframe for subsequent processing.

In [ ]:
combinedf.to_csv('combine_user_venues.csv', index=False)